# Jakość danych 
## Missing, noisy, inconsistent data

In [1]:
import pandas as pd
import numpy as np

### MISSING & NOISY

In [2]:
url = 'https://raw.githubusercontent.com/dataoptimal/posts/master/data%20cleaning%20with%20python%20and%20pandas/property%20data.csv'
#wczytaj zbior jako data.frame korzystajac z pandas
#df = pd.read_csv(url, error_bad_lines=False)
df = pd.read_csv(url)

In [5]:
#skupmy sie na 'OWN_OCCUPIED'. Ile tu jest braków? Ile powinno być?

In [6]:
n=0
for row in df['OWN_OCCUPIED']:
    try:
        int(row)
        df.loc[n, 'OWN_OCCUPIED']=np.nan
    except ValueError:
        pass
    n+=1

#### Jak sobie poradzić z brakami?

In [7]:
#Najprostsze rozwiązanie: usuń kolumny/przypadki. Użyj dropna
df_without_missing_values = df.dropna(axis=1)
df.dropna(inplace=True)
df.dropna(how="all") #usunięcie wierszy, w których wszytskie kolumny mają wartość NaN
df.dropna(thresh=4)  #usunięcie wierszy z co najmniej 4 próbkami
df.dropna(subset=['ST_NAME'])

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3,1,1000
1,100002000.0,197.0,LEXINGTON,N,3,1.5,--
8,100009000.0,215.0,TREMONT,Y,na,2,1800


In [8]:
# Opcja lepsza: imputacja
# Pierwsza propozycja: Imputacja wartością 'mean', 'median', 'most_frequent', strategy='constant' fill_value=value
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values= np.nan, strategy='median')

#wybierz zmienne numeryczne i wykorzystaj 'imputer' do uzupełnienia braków
numeric_columns = df.select_dtypes(include='number').columns
df[numeric_columns] = pd.DataFrame(imputer.fit_transform(df[numeric_columns]), columns=numeric_columns)

In [9]:
#Jeszcze lepsza: k-Nearest Neighbors (KNN Imputer does not recognize text data values, requires to normalize data)
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
imputer = KNNImputer(n_neighbors=2, weights='uniform', metric='nan_euclidean')

In [10]:
#przykład 
from sklearn.metrics.pairwise import nan_euclidean_distances
X = [[3,np.nan, 5]]
Y = [[1, 0, 0]]
nan_euclidean_distances(X,Y)
#plus waga, tj. 3/2 

array([[6.59545298]])

In [11]:
#zastosuj KNN
numeric_columns = df.select_dtypes(include='number').columns
df[numeric_columns] = pd.DataFrame(scaler.fit_transform(df[numeric_columns]), columns=numeric_columns)
df[numeric_columns] = pd.DataFrame(imputer.fit_transform(df[numeric_columns]), columns=numeric_columns)

### Przetwarzanie danych kategoryzujących

In [14]:
df = pd.DataFrame([
            ['Zielony', 'M', 10.1, 'klasa1'], 
            ['Czerwony', 'L', 13.5, 'klasa2'], 
            ['Niebieski', 'XL', 15.3, 'klasa1']])

df.columns = ['Kolor', 'Rozmiar', 'Cena', 'Etykieta klas']
df

,Kolor,Rozmiar,Cena,Etykieta klas
0,Zielony,M,10.1,klasa1
1,Czerwony,L,13.5,klasa2
2,Niebieski,XL,15.3,klasa1


In [15]:
#Mapowanie cech porządkowych
size_mapping = {
           'XL': 3,
           'L': 2,
           'M': 1}

df['Rozmiar'] = df['Rozmiar'].map(size_mapping)

In [16]:
#Kodowanie etykiet klas - etykiety klas to nie cechy porządkowe!

class_mapping = {label:idx for idx,label in enumerate(np.unique(df['Etykieta klas']))} #słownik mapowania
class_mapping
df['Etykieta klas'] = df['Etykieta klas'].map(class_mapping)

### Kodowanie „gorącojedynkowe” cech nominalnych

In [17]:
pd.get_dummies(df[['Cena', 'Kolor', 'Rozmiar']])

,Cena,Rozmiar,Kolor_Czerwony,Kolor_Niebieski,Kolor_Zielony
0,10.1,1,0,0,1
1,13.5,2,1,0,0
2,15.3,3,0,1,0


### Skalowanie cech

In [18]:
df_wine = pd.read_csv('https://raw.githubusercontent.com/rasbt/python-machine-learning-book/master/code/datasets/wine/wine.data', header=None)

df_wine.columns = ['Etykieta klas', 'Alkohol', 'Kwas jabłkowy', 'Popiół', 
'Zasadowość popiołu', 'Magnez', 'Całk. zaw. fenoli', 
'Flawonoidy', 'Fenole nieflawonoidowe', 'Proantocyjaniny', 
'Intensywność koloru', 'Odcień', 'Transmitancja 280/315 nm', 'Prolina']
df_wine.head()

,Etykieta klas,Alkohol,Kwas jabłkowy,Popiół,Zasadowość popiołu,Magnez,Całk. zaw. fenoli,Flawonoidy,Fenole nieflawonoidowe,Proantocyjaniny,Intensywność koloru,Odcień,Transmitancja 280/315 nm,Prolina
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [19]:
from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

if Version(sklearn_version) < '0.18':
    from sklearn.cross_validation import train_test_split
else:
    from sklearn.model_selection import train_test_split

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.3, random_state=0)

In [20]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

In [21]:
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [22]:
ex = pd.DataFrame([0, 1, 2 ,3, 4, 5])

# standaryzacja
ex[1] = (ex[0] - ex[0].mean()) / ex[0].std(ddof=0)

# normalizacja
ex[2] = (ex[0] - ex[0].min()) / (ex[0].max() - ex[0].min())
ex.columns = ['input', 'standardized', 'normalized']
ex

,input,standardized,normalized
0,0,-1.46385,0.0
1,1,-0.87831,0.2
2,2,-0.29277,0.4
3,3,0.29277,0.6
4,4,0.87831,0.8
5,5,1.46385,1.0


### INCONSISTENT

In [23]:
# polecam: https://journal.r-project.org/archive/2014-1/loo.pdf 

In [24]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [25]:
#install fuzzywuzzy 
# wykorzystuje metrykę w przestrzeni ciągu znaków - odległość Levensteina
import fuzzywuzzy
from fuzzywuzzy import process

In [26]:
#ratio
from fuzzywuzzy import fuzz
Str1 = "Apple Inc."
Str2 = "apple Inc"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
print(Ratio)

95


In [27]:
Str1 = "Los Angeles Lakers"
Str2 = "Lakers"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
print(Ratio)
print(Partial_Ratio)

50
100


In [28]:
#token_sort_ratio
Str1 = "united states v. nixon"
Str2 = "Nixon v. United States"
Ratio = fuzz.ratio(Str1.lower(),Str2.lower())
Partial_Ratio = fuzz.partial_ratio(Str1.lower(),Str2.lower())
Token_Sort_Ratio = fuzz.token_sort_ratio(Str1,Str2)
print(Ratio)
print(Partial_Ratio)
print(Token_Sort_Ratio)

59
74
100


In [29]:
#process
from fuzzywuzzy import process
str2Match = "apple inc"
strOptions = ["Apple Inc.","apple park","apple incorporated"]
Ratios = process.extract(str2Match,strOptions)
print(Ratios)
# You can also select the string with the highest matching percentage
highest = process.extractOne(str2Match,strOptions)
print(highest)

[('Apple Inc.', 100), ('apple incorporated', 90), ('apple park', 67)]
('Apple Inc.', 100)
